# Generate Tests for fo_h2o system

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from glob import glob
import pytest

from thermocodegen.testing import Tester

Set a time-stamp and version for the tests


In [ ]:
import thermocodegen as tcg
timestamp = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
VERSION = tcg.__version__

### let's set up some directory names for clarity

In [ ]:
HOME_DIR = os.path.abspath('../')
TEST_DIR = HOME_DIR+'/tests/test-v{}-{}'.format(VERSION,timestamp)
try:
    os.makedirs(TEST_DIR)
except:
    pass
print(TEST_DIR)

### Import the thermocodegen reaction module 
make sure that the module is available in your python path

In [ ]:
# load reaction object
import py_fo_h2o_hydration as pfh
rxn = pfh.fo_h2o_hydration()
rxn.report()

### Create a tester object
and report the phase dictionary

In [ ]:
tester = Tester.factory(pfh)
tester.phase_dict

## Test endmembers
### set input dataframe for endmembers

In [ ]:
em_df_in = tester.get_endmember_df_in()
em_df_in

### just set P,T fields for testing

In [ ]:
# Set test variables
T=273.15+150.
P=10.
em_df_in['P']= P*np.ones(len(em_df_in))
em_df_in['T']= T*np.ones(len(em_df_in))
em_df_in['index'] = 0
em_df_in['value'] = np.array([ 0., -925936.6, -4363356.18, -2174419.92, 0.])
em_df_in.dropna(axis=1,inplace=True)
em_df_in

### Generate output testing dataframe

In [ ]:
em_df_out = tester.set_df_out(em_df_in)
em_df_out

### Output dataframe as csv file and pytest files to TEST_DIR

In [ ]:
em_df_out.to_csv('{}/Endmembers.csv'.format(TEST_DIR))

In [ ]:
fileroot='{}/test_endmembers'.format(TEST_DIR) 
tester.write_tests(em_df_out,fileroot=fileroot)

### run pytest on the test files

In [ ]:
file = glob(fileroot+'*')[-1]
pytest.main(['--disable-warnings',file])

## Test Phases
### set input dataframe for phases

In [ ]:
ph_df_in = tester.get_phase_df_in()
ph_df_in

### just set P,T,c,x,n fields for testing

In [ ]:
rxn.report()

In [ ]:
C=rxn.zero_C()
for i in range(len(C)):
    C[i] = [1.0]
C

In [ ]:
X = rxn.C_to_X(C)

In [ ]:
phases = rxn.phases()
E_dict = { phase.name(): phase.conv_moles_to_elm([1.]) for phase in phases }

In [ ]:
ph_df_in['P']= P*np.ones(len(C))
ph_df_in['T']= T*np.ones(len(C))
ph_df_in['index'] = 0
ph_df_in['value'] = 298.15*np.ones(len(C))
for index,row in ph_df_in.iterrows():
    name = row['name']
    print(index,name)
    ph_df_in['c'][index] = np.array(C[index])
    ph_df_in['x'][index] = np.array(X[index])
    ph_df_in['n'][index] = np.array(X[index])
    ph_df_in['e'][index] = np.array(E_dict[name])
ph_df_in.dropna(axis=1,inplace=True)
ph_df_in

### Generate output testing dataframe and pytest files

In [ ]:
ph_df_out = tester.set_df_out(ph_df_in)
ph_df_out

In [ ]:
ph_df_out.to_csv('{}/Phases.csv'.format(TEST_DIR))

In [ ]:
fileroot='{}/test_phases'.format(TEST_DIR) 
tester.write_tests(ph_df_out,fileroot=fileroot)

### run pytest on the test files

In [ ]:
file=glob(fileroot+'*')[-1]
pytest.main(['--disable-warnings',file])

## Test Reactions
### set input dataframe for reactions

In [ ]:
rx_df_in = tester.get_reaction_df_in()
rx_df_in

### just set P,T,c,x,n fields for testing

In [ ]:
rxn.report()

In [ ]:
# Set test variables
X = rxn.C_to_X(C)
# set phase fractions for the phases
Phi = [0.9, 0.0, 0.0, 0.1]

# Variables for parameter set and get functions
p, val = "R", 8.314

#jacobian terms
j = 0
i = 1
l = 0
k = 0
m = 1

In [ ]:
rx_df_in['P']= P
rx_df_in['T']= T
rx_df_in['C']= [C]
rx_df_in['X']= [X]
rx_df_in['Phi'] = [Phi]
rx_df_in[['i','j','k','l','m']]=pd.DataFrame.from_dict(dict(i=[i],j=[j],k=[k],l=[l],m=[m]))
rx_df_in['p'] = p
rx_df_in['val'] = val
rx_df_in


### Generate output testing dataframe and pytest files

In [ ]:
rx_df_out = tester.set_df_out(rx_df_in)
rx_df_out

In [ ]:
rx_df_out.to_csv('{}/Reactions.csv'.format(TEST_DIR))

In [ ]:
fileroot='{}/test_rxns'.format(TEST_DIR)
tester.write_tests(rx_df_out,fileroot=fileroot)

In [ ]:
file = glob(fileroot+'*')[-1]
pytest.main(['--disable-warnings',file])